<a href="https://colab.research.google.com/github/prashanthmoravineni/deepfake_detection/blob/main/Deepfake_Artifact_Detection_using_texture_and_frequency_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive"

 0d89715b-ed58-45a2-8323-cede02722870.pptx
'22eg106a40(ai-a) dwv_pbl_activity (1).gdoc'
'22eg106a40(ai-a) dwv_pbl_activity.gdoc'
'22eg106a40(ai-a) dwv_pbl_activity.pdf'
 22eg106a41
'22eg106a41 (1).pdf'
'22eg106a41 (2).pdf'
'22EG106a41(M.PRASHANTH).pdf'
 22eg106a41.pdf
 22eg106a41_prashanth.pdf
 22EG106A41.zip
'6ac3b3fc-be8d-466e-89c8-8c56aa664a0a (1).gdoc'
'6ac3b3fc-be8d-466e-89c8-8c56aa664a0a (1).pdf'
'6ac3b3fc-be8d-466e-89c8-8c56aa664a0a (1).pptx'
'6ac3b3fc-be8d-466e-89c8-8c56aa664a0a (2).gdoc'
 6ac3b3fc-be8d-466e-89c8-8c56aa664a0a.gdoc
 6ac3b3fc-be8d-466e-89c8-8c56aa664a0a.pdf
 6ac3b3fc-be8d-466e-89c8-8c56aa664a0a.pptx
'a42 w-6 eml.gdoc'
'a42 w-6 eml.pdf'
'AICTE CERTIFICATE _22EG106A41_PRASHANTH (1).PDF'
'AICTE CERTIFICATE _22EG106A41_PRASHANTH (2).PDF'
'AICTE CERTIFICATE _22EG106A41_PRASHANTH.PDF'
'Applicant_List_for_2026_Cognizant_On-Campus_hiring_Aug-_Sept_at_Cognizant (1).xlsx'
 Assessment.gform
'bhavishyakala external.pdf'
'CarRentalSystem1(22EG106A41).zip'
 CarRentalSystem1.zi

In [ ]:
!ls "/content/drive/MyDrive/deepfake.zip"


/content/drive/MyDrive/deepfake.zip


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
zip_path = "/content/drive/MyDrive/deepfake.zip"
extract_path = "/content/dataset"
print("⏳ Extracting dataset...")
!unzip -q -o "$zip_path" -d "$extract_path"
print("✅ Extraction complete\n")
real_count, fake_count = 0, 0
for root, dirs, files in os.walk(extract_path):
    for f in files:
        f_lower = f.lower()
        if f_lower.endswith((".jpg", ".png", ".jpeg", ".mp4", ".avi", ".mov")):
            if "real" in root.lower() or "real" in f_lower:
                real_count += 1
            elif "fake" in root.lower() or "fake" in f_lower:
                fake_count += 1
print("📊 Dataset Summary:")
print(f"Real files : {real_count}")
print(f"Fake files : {fake_count}")
print(f"Total      : {real_count + fake_count}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Extracting dataset...
✅ Extraction complete

📊 Dataset Summary:
Real files : 24031
Fake files : 24029
Total      : 48060


In [ ]:
!pip install scikit-image tensorflow matplotlib opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
import cv2, numpy as np, tensorflow as tf
from skimage.feature import local_binary_pattern
from keras.utils import to_categorical

def preprocess_image(img_path, size=128):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    img = cv2.resize(img, (size, size))

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # --- FFT ---
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    magnitude_spectrum = cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX)

    # --- LBP ---
    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    lbp = cv2.normalize(lbp, None, 0, 255, cv2.NORM_MINMAX)

    stacked = np.dstack([gray, magnitude_spectrum, lbp]).astype(np.float32) / 255.0
    return stacked

In [ ]:
!pip install opencv-python scikit-image tensorflow matplotlib -q

import os, cv2, numpy as np, tensorflow as tf
from skimage.feature import local_binary_pattern

# -------------------------------
# Preprocessing function
# -------------------------------
def preprocess_image(img_path, size=128):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    img = cv2.resize(img, (size, size))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # FFT
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    magnitude_spectrum = cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX)

    # LBP
    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    lbp = cv2.normalize(lbp, None, 0, 255, cv2.NORM_MINMAX)

    stacked = np.dstack([gray, magnitude_spectrum, lbp]).astype(np.float32) / 255.0
    return stacked

# -------------------------------
# Data Generator
# -------------------------------
def data_generator(dataset_path, batch_size=32, size=128):
    real_paths, fake_paths = [], []
    for root, _, files in os.walk(dataset_path):
        for f in files:
            if f.lower().endswith((".jpg",".png",".jpeg")):
                fpath = os.path.join(root, f)
                if "real" in root.lower():
                    real_paths.append((fpath, 0))
                elif "fake" in root.lower():
                    fake_paths.append((fpath, 1))

    all_data = real_paths + fake_paths
    print(f"📊 Found {len(all_data)} images (Real: {len(real_paths)}, Fake: {len(fake_paths)})")

    while True:
        np.random.shuffle(all_data)
        for i in range(0, len(all_data), batch_size):
            batch = all_data[i:i+batch_size]
            imgs, labels = [], []
            for fpath, label in batch:
                processed = preprocess_image(fpath, size)
                if processed is not None:
                    imgs.append(processed)
                    labels.append(label)

            if imgs:
                yield np.array(imgs), tf.keras.utils.to_categorical(labels, num_classes=2)

# -------------------------------
# Test Run
# -------------------------------
dataset_path = "/content/dataset"
gen = data_generator(dataset_path, batch_size=32, size=128)
X, y = next(gen)
print("🔍 First batch shape:", X.shape, y.shape)


📊 Found 48060 images (Real: 24031, Fake: 24029)
🔍 First batch shape: (32, 128, 128, 3) (32, 2)


In [ ]:
import os, cv2, numpy as np, tensorflow as tf
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split

# ---------- Preprocessing ----------
def preprocess_image(img_path, size=128):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    img = cv2.resize(img, (size, size))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # FFT
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    magnitude_spectrum = cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX)

    # LBP
    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    lbp = cv2.normalize(lbp, None, 0, 255, cv2.NORM_MINMAX)

    stacked = np.dstack([gray, magnitude_spectrum, lbp]).astype(np.float32) / 255.0
    return stacked

# ---------- Collect Paths ----------
dataset_path = "/content/dataset"
real_paths, fake_paths = [], []
for root, _, files in os.walk(dataset_path):
    for f in files:
        if f.lower().endswith((".jpg",".jpeg",".png")):
            fpath = os.path.join(root, f)
            if "real" in root.lower():
                real_paths.append((fpath, 0))
            elif "fake" in root.lower():
                fake_paths.append((fpath, 1))

all_data = real_paths + fake_paths
print(f"📊 Found {len(all_data)} images")

# ---------- Train-Test Split ----------
paths, labels = zip(*all_data)
train_paths, test_paths, y_train, y_test = train_test_split(paths, labels, test_size=0.2, random_state=42)

# ---------- Generator ----------
def data_generator(file_paths, labels, batch_size=32, size=128):
    while True:
        idx = np.random.permutation(len(file_paths))
        for i in range(0, len(file_paths), batch_size):
            batch_idx = idx[i:i+batch_size]
            imgs, lbls = [], []
            for j in batch_idx:
                img = preprocess_image(file_paths[j], size)
                if img is not None:
                    imgs.append(img)
                    lbls.append(labels[j])
            if imgs:
                yield np.array(imgs), tf.keras.utils.to_categorical(lbls, num_classes=2)

# ---------- Test Run ----------
train_gen = data_generator(train_paths, y_train, batch_size=16, size=128)
Xb, yb = next(train_gen)
print("✅ First batch shape:", Xb.shape, yb.shape)

📊 Found 48060 images
✅ First batch shape: (16, 128, 128, 3) (16, 2)


In [ ]:
!pip install tensorflow opencv-python scikit-image tabulate -q
import tensorflow as tf, os
from tensorflow.keras import layers, models
from tabulate import tabulate

# ==========================
# 1. PATH + CONFIG
# ==========================
dataset_path = "/content/dataset"
img_size = (64, 64)
batch_size = 128
epochs = 5

# ==========================
# 2. DATA PIPELINE
# ==========================
def process_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    img = tf.cast(img, tf.float32) / 255.0
    return img, tf.one_hot(label, 2)

all_files, all_labels = [], []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith((".jpg",".jpeg",".png")):
            fpath = os.path.join(root, file)
            if "real" in fpath.lower():
                all_files.append(fpath); all_labels.append(0)
            elif "fake" in fpath.lower():
                all_files.append(fpath); all_labels.append(1)

print(f"🔍 Total images: {len(all_files)}")

# Train/Test split
train_size = int(0.8 * len(all_files))
file_ds = tf.data.Dataset.from_tensor_slices((all_files, all_labels))
dataset = file_ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = dataset.take(train_size).shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds   = dataset.skip(train_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

print(f"✅ Train samples: {train_size}, Validation: {len(all_files)-train_size}")

# ==========================
# 3. MODEL
# ==========================
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(64,64,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation="relu"),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(2, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# ==========================
# 4. TRAIN + RESULTS TABLE
# ==========================
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=1)

# Format history into table
results = []
for i in range(epochs):
    results.append([
        i+1,
        round(history.history['accuracy'][i], 4),
        round(history.history['val_accuracy'][i], 4),
        round(history.history['loss'][i], 4),
        round(history.history['val_loss'][i], 4)
    ])

print("\n📊 Training Summary:")
print(tabulate(results, headers=["Epoch", "Train Acc", "Val Acc", "Train Loss", "Val Loss"], tablefmt="grid"))


🔍 Total images: 48060
✅ Train samples: 38448, Validation: 9612
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


301/301 ━━━━━━━━━━━━━━━━━━━━ 234s 769ms/step - accuracy: 0.7407 - loss: 0.4978 - val_accuracy: 0.9737 - val_loss: 0.1316
Epoch 2/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 223s 738ms/step - accuracy: 0.8170 - loss: 0.3960 - val_accuracy: 0.9576 - val_loss: 0.1424
Epoch 3/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 225s 746ms/step - accuracy: 0.8543 - loss: 0.3227 - val_accuracy: 0.9692 - val_loss: 0.1050
Epoch 4/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 223s 739ms/step - accuracy: 0.8862 - loss: 0.2683 - val_accuracy: 0.9543 - val_loss: 0.1286
Epoch 5/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 221s 733ms/step - accuracy: 0.9026 - loss: 0.2277 - val_accuracy: 0.9813 - val_loss: 0.0622

📊 Training Summary:
+---------+-------------+-----------+--------------+------------+
|   Epoch |   Train Acc |   Val Acc |   Train Loss |   Val Loss |
+=========+=============+===========+==============+============+
|       1 |      0.8225 |    0.9737 |       0.3861 |     0.1316 |
+---------+-------------+-----------+--------------+------------+
|   

In [ ]:
!pip install opencv-python tensorflow -q
from google.colab import files
import numpy as np
import cv2
import tensorflow as tf

# ==========================
# IMAGE PREPROCESS FUNCTION
# ==========================
def preprocess_image(img_path, img_size=(64,64)):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"❌ Could not read image at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img.astype("float32") / 255.0   # ensure float32
    return img

# ==========================
# PREDICT FUNCTION
# ==========================
def predict_image(img_path):
    processed = preprocess_image(img_path)
    processed = np.expand_dims(processed, axis=0).astype("float32")  # shape (1,64,64,3)
    pred = model.predict(processed, verbose=0)[0]
    label = "FAKE" if np.argmax(pred) == 1 else "REAL"
    confidence = round(100 * np.max(pred), 2)
    return label, confidence

# ==========================
# UPLOAD + TEST
# ==========================
uploaded = files.upload()   # Upload an image from your system
for filename in uploaded.keys():
    label, conf = predict_image(filename)
    print(f"✅ Prediction for {filename}: {label} ({conf}% confidence)")

Saving WhatsApp Image 2025-09-10 at 11.17.01_9750ef46.jpg to WhatsApp Image 2025-09-10 at 11.17.01_9750ef46.jpg
✅ Prediction for WhatsApp Image 2025-09-10 at 11.17.01_9750ef46.jpg: REAL (78.55000305175781% confidence)
